In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd
import os

In [ ]:
#Create an instance of ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
class_labels = {'cocci': 0, 'bacilli': 1, 'spirilla': 2}

# Load training data from the 'train' folder
# Each subfolder (bacilli, cocci, spirilla) represents a class
train_data = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Bacterial Classification/train', # Path to the train folder
    target_size=(224, 224),       # Resize all images to 224x224
    batch_size=32,                # Number of images per batch
    class_mode='categorical',      # Multi-class classification
    classes=class_labels          # Explicit class mapping

)

# Load validation data from the 'validation' folder
# Each subfolder (bacilli, cocci, spirilla) represents a class
val_data = val_datagen.flow_from_directory(
    '/content/drive/MyDrive/Bacterial Classification/validation',# Path to the validation folder
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    classes=class_labels

)

# Check class mappings
print("Training Class Indices:", train_data.class_indices)
print("Validation Class Indices:", val_data.class_indices)


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze the base model

model = tf.keras.Sequential([
    base_model,
    GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(
    optimizer=Adam(learning_rate=0.0001),  # Lower learning rate
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)
# Train the model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=50,                # Allow more epochs but stop early if needed
    callbacks=[early_stopping]
)


# Evaluate the model on the validation dataset
val_loss, val_accuracy = model.evaluate(val_data)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


12/12 ━━━━━━━━━━━━━━━━━━━━ 136s 7s/step - accuracy: 0.3350 - loss: 1.6972 - val_accuracy: 0.3417 - val_loss: 1.3020
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 182ms/step - accuracy: 0.3816 - loss: 1.3227 - val_accuracy: 0.4750 - val_loss: 1.1209
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 6s 223ms/step - accuracy: 0.5357 - loss: 0.9564 - val_accuracy: 0.5583 - val_loss: 1.0034
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 175ms/step - accuracy: 0.5961 - loss: 0.8981 - val_accuracy: 0.5667 - val_loss: 0.9151
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 226ms/step - accuracy: 0.5730 - loss: 0.9111 - val_accuracy: 0.5833 - val_loss: 0.8556
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 184ms/step - accuracy: 0.7188 - loss: 0.6853 - val_accuracy: 0.6333 - val_loss: 0.8078
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 218ms/step - accuracy: 0.7019 - loss: 0.6919 - val_accuracy: 0.6750 - val_loss: 0.7685
Epoch 8/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 236ms/step - accuracy: 0.7730 - loss: 0.5996 - val_accuracy: 0.6833 - val_l

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import load_img, img_to_array

# Load the file containing test image names
test_images = pd.read_csv('/content/drive/MyDrive/Bacterial Classification/test_filenames.txt', header=None)
test_images.columns = ['Image Name']

# Path to the test folder containing the images
test_dir = '/content/drive/MyDrive/Bacterial Classification/test'

# Placeholder for predictions
predictions = []

# Process each image and predict
for img_name in test_images['Image Name']:
    # Construct the full path to the image
    img_path = os.path.join(test_dir, img_name)

    # Load and preprocess the image
    img = load_img(img_path, target_size=(224, 224))  # Resize image to match the model's input size
    img_array = img_to_array(img) / 255.0            # Normalize pixel values
    img_array = np.expand_dims(img_array, axis=0)    # Add batch dimension

    # Make a prediction using the trained model
    prediction = model.predict(img_array, verbose=0)  # Suppress verbose output
    predictions.append(prediction.argmax())          # Append the predicted class index (0, 1, 2)

# Add predictions to the DataFrame
test_images['Predicted Class'] = predictions

In [ ]:
predictions

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 2,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2]

In [ ]:
model.save('/content/drive/MyDrive/Bacterial Classification/saved_model.keras')